In [1]:
import numpy as np
import pandas as pd
from collections import OrderedDict
import matplotlib.pyplot as plt
import seaborn as sns

In [44]:
# data from the Ultrascale Systems Research Center (USRC) in Los Alamos
# 'This is failure/interrupt data for all systems from 1996 to 2005'

df = pd.read_csv('data/LA-UR-05-7318-failure-data-1996-2005_3.csv')
df.head(2)

,System,machine type,nodes,procstot,procsinnode,nodenum,nodenumz,node install,node prod,node decom,...,Prob Started (mm/dd/yy hh:mm),Prob Fixed (mm/dd/yy hh:mm),Down Time,Facilities,Hardware,Human Error,Network,Undetermined,Software,Same Event
0,2,cluster,49.0,6152.0,80.0,0.0,0.0,2023-04-05,2023-06-05,current,...,6/21/05 10:54,6/21/05 11:00,6,NaN,Graphics Accel Hdwr,NaN,NaN,NaN,NaN,No
1,2,cluster,49.0,6152.0,80.0,0.0,0.0,2023-04-05,2023-06-05,current,...,9/6/05 9:13,9/6/05 9:19,6,NaN,NaN,NaN,NaN,NaN,Other Software,No


In [45]:
# rename column names
df = df.rename(columns={
    'machine type':'machine_type',
    'node install':'node_installed_date',
    'node prod':'node_in_production_date',
    'node decom':'node_decommissioned_date',
    'fru type':'fru_type',
    'num intercon':'num_interconnections',
    'Prob Started (mm/dd/yy hh:mm)':'problem_start_datetime',
    'Prob Fixed (mm/dd/yy hh:mm)':'problem_end_datetime',
    'Down Time':'down_time_mins',
    'Human Error':'human_error',
    'Same Event':'is_same_event'
})
df.head(2)

,System,machine_type,nodes,procstot,procsinnode,nodenum,nodenumz,node_installed_date,node_in_production_date,node_decommissioned_date,...,problem_start_datetime,problem_end_datetime,down_time_mins,Facilities,Hardware,human_error,Network,Undetermined,Software,is_same_event
0,2,cluster,49.0,6152.0,80.0,0.0,0.0,2023-04-05,2023-06-05,current,...,6/21/05 10:54,6/21/05 11:00,6,NaN,Graphics Accel Hdwr,NaN,NaN,NaN,NaN,No
1,2,cluster,49.0,6152.0,80.0,0.0,0.0,2023-04-05,2023-06-05,current,...,9/6/05 9:13,9/6/05 9:19,6,NaN,NaN,NaN,NaN,NaN,Other Software,No


In [46]:
df.isna().sum()

System                          0
machine_type                  126
nodes                         126
procstot                      126
procsinnode                   126
nodenum                       126
nodenumz                      126
node_installed_date           126
node_in_production_date       126
node_decommissioned_date      126
fru_type                      126
mem                           126
cputype                       126
memtype                       126
num_interconnections          126
purpose                       126
problem_start_datetime          0
problem_end_datetime            0
down_time_mins                  0
Facilities                  23377
Hardware                     9398
human_error                 23590
Network                     23318
Undetermined                21087
Software                    18378
is_same_event                   0
dtype: int64

In [47]:
# drop rows where machine_type isna
df = df[df['machine_type'].notna()]
df.isna().sum()

System                          0
machine_type                    0
nodes                           0
procstot                        0
procsinnode                     0
nodenum                         0
nodenumz                        0
node_installed_date             0
node_in_production_date         0
node_decommissioned_date        0
fru_type                        0
mem                             0
cputype                         0
memtype                         0
num_interconnections            0
purpose                         0
problem_start_datetime          0
problem_end_datetime            0
down_time_mins                  0
Facilities                  23256
Hardware                     9322
human_error                 23464
Network                     23193
Undetermined                20972
Software                    18311
is_same_event                   0
dtype: int64

In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23613 entries, 0 to 23738
Data columns (total 26 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   System                    23613 non-null  int64  
 1   machine_type              23613 non-null  object 
 2   nodes                     23613 non-null  float64
 3   procstot                  23613 non-null  float64
 4   procsinnode               23613 non-null  float64
 5   nodenum                   23613 non-null  float64
 6   nodenumz                  23613 non-null  float64
 7   node_installed_date       23613 non-null  object 
 8   node_in_production_date   23613 non-null  object 
 9   node_decommissioned_date  23613 non-null  object 
 10  fru_type                  23613 non-null  object 
 11  mem                       23613 non-null  float64
 12  cputype                   23613 non-null  float64
 13  memtype                   23613 non-null  float64
 14  num_in

In [49]:
df.node_installed_date.unique()

array(['2023-04-05', '1996-11-01', '2023-08-03', 'before tracking',
       '2023-03-01', '2023-12-02', '2023-03-05', '2023-11-04',
       '1996-10-01', '2023-03-02', '2023-08-02', '2023-10-01',
       '2023-08-01', '1998-01-01', '1998-10-01', '2023-11-02'],
      dtype=object)

In [50]:
# replace rows where node_installed_date == 'before tracking' with MIN date
df.loc[df["node_installed_date"] == "before tracking", "node_installed_date"] = '1996-10-01'
df.node_installed_date.unique()

array(['2023-04-05', '1996-11-01', '2023-08-03', '1996-10-01',
       '2023-03-01', '2023-12-02', '2023-03-05', '2023-11-04',
       '2023-03-02', '2023-08-02', '2023-10-01', '2023-08-01',
       '1998-01-01', '1998-10-01', '2023-11-02'], dtype=object)

In [51]:
df.node_in_production_date.unique()

array(['2023-06-05', '1997-01-01', '2023-09-03', 'before tracking',
       '2023-04-01', '2023-12-02', '2023-03-05', '2023-11-04',
       '1996-12-01', '2023-05-02', '2023-10-02', '2023-12-01',
       '2023-09-01', '1998-01-01', '1998-10-01', '2023-11-02'],
      dtype=object)

In [52]:
# replace rows where node_in_production_date == 'before tracking' w/ MIN date
df.loc[df["node_in_production_date"] == "before tracking", "node_in_production_date"] = '1996-12-01'
df.node_in_production_date.unique()

array(['2023-06-05', '1997-01-01', '2023-09-03', '1996-12-01',
       '2023-04-01', '2023-12-02', '2023-03-05', '2023-11-04',
       '2023-05-02', '2023-10-02', '2023-12-01', '2023-09-01',
       '1998-01-01', '1998-10-01', '2023-11-02'], dtype=object)

In [53]:
df.node_decommissioned_date.unique()

array(['current', '2023-11-05', '1999-12-01', '2023-06-05', '2023-09-02',
       '2023-01-02', '2023-04-03', '2023-12-04', '2023-12-03'],
      dtype=object)

In [55]:
# replace rows where node_decommissioned_date == 'before tracking' w/ MAX date
df.loc[df["node_decommissioned_date"] == "current", "node_decommissioned_date"] = '2023-12-04'
df.node_decommissioned_date.unique()

array(['2023-12-04', '2023-11-05', '1999-12-01', '2023-06-05',
       '2023-09-02', '2023-01-02', '2023-04-03', '2023-12-03'],
      dtype=object)

In [59]:
#  convert dates to datetime
df['problem_start_datetime'] = pd.to_datetime(df['problem_start_datetime'])
df['problem_end_datetime'] = pd.to_datetime(df['problem_end_datetime'])
df['node_installed_date'] = pd.to_datetime(df['node_installed_date'])
df['node_in_production_date'] = pd.to_datetime(df['node_in_production_date'])
df['node_decommissioned_date'] = pd.to_datetime(df['node_decommissioned_date'])
df.head(2)

,System,machine_type,nodes,procstot,procsinnode,nodenum,nodenumz,node_installed_date,node_in_production_date,node_decommissioned_date,...,problem_start_datetime,problem_end_datetime,down_time_mins,Facilities,Hardware,human_error,Network,Undetermined,Software,is_same_event
0,2,cluster,49.0,6152.0,80.0,0.0,0.0,2023-04-05,2023-06-05,2023-12-04,...,2005-06-21 10:54:00,2005-06-21 11:00:00,6,NaN,Graphics Accel Hdwr,NaN,NaN,NaN,NaN,No
1,2,cluster,49.0,6152.0,80.0,0.0,0.0,2023-04-05,2023-06-05,2023-12-04,...,2005-09-06 09:13:00,2005-09-06 09:19:00,6,NaN,NaN,NaN,NaN,NaN,Other Software,No


In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23613 entries, 0 to 23738
Data columns (total 26 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   System                    23613 non-null  int64         
 1   machine_type              23613 non-null  object        
 2   nodes                     23613 non-null  float64       
 3   procstot                  23613 non-null  float64       
 4   procsinnode               23613 non-null  float64       
 5   nodenum                   23613 non-null  float64       
 6   nodenumz                  23613 non-null  float64       
 7   node_installed_date       23613 non-null  datetime64[ns]
 8   node_in_production_date   23613 non-null  datetime64[ns]
 9   node_decommissioned_date  23613 non-null  datetime64[ns]
 10  fru_type                  23613 non-null  object        
 11  mem                       23613 non-null  float64       
 12  cputype           

In [67]:
# convert float64 columns to int64
df2 = df.copy()
float_col = df2.select_dtypes(include=['float64'])
for col in float_col.columns.values:
    df2[col] = df2[col].astype('int64')
    
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23613 entries, 0 to 23738
Data columns (total 26 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   System                    23613 non-null  int64         
 1   machine_type              23613 non-null  object        
 2   nodes                     23613 non-null  int64         
 3   procstot                  23613 non-null  int64         
 4   procsinnode               23613 non-null  int64         
 5   nodenum                   23613 non-null  int64         
 6   nodenumz                  23613 non-null  int64         
 7   node_installed_date       23613 non-null  datetime64[ns]
 8   node_in_production_date   23613 non-null  datetime64[ns]
 9   node_decommissioned_date  23613 non-null  datetime64[ns]
 10  fru_type                  23613 non-null  object        
 11  mem                       23613 non-null  int64         
 12  cputype           

In [68]:
df2.head()

,System,machine_type,nodes,procstot,procsinnode,nodenum,nodenumz,node_installed_date,node_in_production_date,node_decommissioned_date,...,problem_start_datetime,problem_end_datetime,down_time_mins,Facilities,Hardware,human_error,Network,Undetermined,Software,is_same_event
0,2,cluster,49,6152,80,0,0,2023-04-05,2023-06-05,2023-12-04,...,2005-06-21 10:54:00,2005-06-21 11:00:00,6,NaN,Graphics Accel Hdwr,NaN,NaN,NaN,NaN,No
1,2,cluster,49,6152,80,0,0,2023-04-05,2023-06-05,2023-12-04,...,2005-09-06 09:13:00,2005-09-06 09:19:00,6,NaN,NaN,NaN,NaN,NaN,Other Software,No
2,2,cluster,49,6152,80,0,0,2023-04-05,2023-06-05,2023-12-04,...,2005-09-06 10:32:00,2005-09-06 10:46:00,14,NaN,NaN,NaN,NaN,Undetermined,NaN,No
3,2,cluster,49,6152,80,0,0,2023-04-05,2023-06-05,2023-12-04,...,2005-09-06 14:50:00,2005-09-06 15:08:00,18,NaN,NaN,NaN,NaN,NaN,Other Software,No
4,2,cluster,49,6152,80,0,0,2023-04-05,2023-06-05,2023-12-04,...,2005-09-08 16:52:00,2005-09-08 16:57:00,5,NaN,NaN,NaN,NaN,Undetermined,NaN,No


In [72]:
df3 = df2.copy()
float_col = df3.select_dtypes(include=['object'])
for col in float_col.columns.values:
    df3[col] = df3[col].astype('string')
    
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23613 entries, 0 to 23738
Data columns (total 26 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   System                    23613 non-null  int64         
 1   machine_type              23613 non-null  string        
 2   nodes                     23613 non-null  int64         
 3   procstot                  23613 non-null  int64         
 4   procsinnode               23613 non-null  int64         
 5   nodenum                   23613 non-null  int64         
 6   nodenumz                  23613 non-null  int64         
 7   node_installed_date       23613 non-null  datetime64[ns]
 8   node_in_production_date   23613 non-null  datetime64[ns]
 9   node_decommissioned_date  23613 non-null  datetime64[ns]
 10  fru_type                  23613 non-null  string        
 11  mem                       23613 non-null  int64         
 12  cputype           

In [73]:
df3.head()

,System,machine_type,nodes,procstot,procsinnode,nodenum,nodenumz,node_installed_date,node_in_production_date,node_decommissioned_date,...,problem_start_datetime,problem_end_datetime,down_time_mins,Facilities,Hardware,human_error,Network,Undetermined,Software,is_same_event
0,2,cluster,49,6152,80,0,0,2023-04-05,2023-06-05,2023-12-04,...,2005-06-21 10:54:00,2005-06-21 11:00:00,6,<NA>,Graphics Accel Hdwr,<NA>,<NA>,<NA>,<NA>,No
1,2,cluster,49,6152,80,0,0,2023-04-05,2023-06-05,2023-12-04,...,2005-09-06 09:13:00,2005-09-06 09:19:00,6,<NA>,<NA>,<NA>,<NA>,<NA>,Other Software,No
2,2,cluster,49,6152,80,0,0,2023-04-05,2023-06-05,2023-12-04,...,2005-09-06 10:32:00,2005-09-06 10:46:00,14,<NA>,<NA>,<NA>,<NA>,Undetermined,<NA>,No
3,2,cluster,49,6152,80,0,0,2023-04-05,2023-06-05,2023-12-04,...,2005-09-06 14:50:00,2005-09-06 15:08:00,18,<NA>,<NA>,<NA>,<NA>,<NA>,Other Software,No
4,2,cluster,49,6152,80,0,0,2023-04-05,2023-06-05,2023-12-04,...,2005-09-08 16:52:00,2005-09-08 16:57:00,5,<NA>,<NA>,<NA>,<NA>,Undetermined,<NA>,No


In [ ]:
df.to_csv('data/0_all_systems.csv', sep=',', encoding='utf-8')